<h1>4장 텍스트 분류</h1>
<i>표현 모델과 생성 모델을 사용해 텍스트 분류하기</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter04.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 4장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---


In [1]:
%%capture
# %%capture 매직 명령어는 셀에서 출력되는 모든 내용을 숨겨준다.
# 즉, 설치 과정에서 나오는 긴 메시지들을 화면에 보여주지 않도록 한다.
# 출력이 지저분해지는 것을 막기 위해 주로 사용한다.

!pip install datasets
# pip install datasets : 'datasets'라는 파이썬 라이브러리를 설치하는 명령어.
# 'datasets' 라이브러리는 허깅페이스에서 제공하며,
# 다양한 텍스트/이미지 데이터셋을 쉽게 불러올 수 있게 도와준다.

In [2]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

# 영화 리뷰 데이터셋

In [3]:
from datasets import load_dataset
# load_dataset 함수는 허깅페이스의 datasets 라이브러리에서
# 다양한 공개 데이터셋을 쉽게 불러올 수 있게 해주는 함수이다.

# 데이터를 로드합니다.
data = load_dataset("rotten_tomatoes")
# "rotten_tomatoes"는 영화 리뷰 감정분석(Sentiment Analysis)에 자주 사용되는 데이터셋이다.
# 긍정/부정 리뷰가 포함되어 있으며, 텍스트 분류 실습에 많이 쓰인다.
# load_dataset 함수를 호출하면 학습(train), 검증(validation), 테스트(test) 세트로 자동 분리되어 로드된다.

data
# 불러온 데이터셋 전체 정보를 출력해서 구조를 확인한다.
# 각 split에 어떤 데이터가 들어 있는지 확인할 수 있다.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [4]:
data["train"][0, -1]
# data["train"] : 학습용(train) 데이터셋을 의미한다.
# [0, -1]       : 첫 번째 샘플의 마지막 컬럼을 선택한다는 뜻이다.

# 즉, "train" 데이터 중에서 첫 번째 데이터의 마지막 항목을 가져온다.
# Rotten Tomatoes 데이터셋에서는 일반적으로 마지막 값이 '레이블(label)'이다.
# label은 리뷰가 긍정(1)인지, 부정(0)인지 표시하는 숫자이다.

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

# 표현 모델로 텍스트 분류하기

# 작업에 특화된 모델 사용하기

In [5]:
from transformers import pipeline
# transformers 라이브러리에서 pipeline 기능을 불러온다.
# pipeline은 복잡한 모델 설정 없이도 바로 모델을 불러와 사용할 수 있게 해주는 편리한 기능이다.

# 허깅 페이스 모델 경로
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
# 사용할 사전학습 모델의 이름(또는 경로)이다.
# 이 모델은 '트위터 감정 분석'에 특화된 RoBERTa 기반 모델이다.

# 파이프라인으로 모델을 로드합니다.
pipe = pipeline(
    model=model_path,         # 사용할 모델 파일
    tokenizer=model_path,     # 해당 모델에 맞는 토크나이저(문장을 숫자로 변환)
    return_all_scores=True,   # 감정 점수를 모두 반환 (긍정/부정 등 전체 점수)
    device="cuda:0"           # GPU 0번 장치를 사용하겠다는 뜻 (GPU 사용 시 속도가 매우 빨라진다)
)
# pipeline 함수를 이용하면 'pipe'라는 하나의 객체로 쉽게 감정 분석을 수행할 수 있다.
# 예: pipe("I love this movie!") 이렇게 바로 사용할 수 있다.

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return

In [6]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# 예측 결과를 저장할 리스트
y_pred = []

# 감정 분석 파이프라인(pipe)을 이용해 테스트 데이터 전체에 대해 추론을 실행한다.
# KeyDataset은 datasets 객체에서 특정 컬럼만 꺼내서 pipeline에 넣을 수 있도록 도와주는 유틸리티이다.
# 여기서는 data["test"] 안의 "text" 컬럼만 뽑아서 모델에 전달한다.

for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    # output은 감정 분석 모델이 반환하는 결과이며,
    # 각 감정(label)별 점수가 리스트 형태로 들어 있다.
    # 예: [{'label': 'NEGATIVE', 'score': 0.8}, {'label': 'NEUTRAL', 'score': ...}, {'label': 'POSITIVE', 'score': ...}]

    negative_score = output[0]["score"]  # 부정(NEGATIVE) 점수
    positive_score = output[2]["score"]  # 긍정(POSITIVE) 점수
    # ※ 카드립 모델은 label 순서가 [NEGATIVE, NEUTRAL, POSITIVE] 형태이므로 index 0, 2를 사용한다.

    # np.argmax는 리스트에서 가장 큰 값을 가진 인덱스를 반환한다.
    # 즉, 부정/긍정 중 어떤 점수가 더 높은지를 선택하는 과정이다.
    assignment = np.argmax([negative_score, positive_score])

    # 예측 결과를 저장 (0 = 부정, 1 = 긍정)
    y_pred.append(assignment)

100%|██████████| 1066/1066 [00:11<00:00, 94.79it/s] 


In [7]:
from sklearn.metrics import classification_report
# classification_report는 모델이 얼마나 잘 예측했는지
# 정밀도(precision), 재현율(recall), f1-score 등을 표로 보여주는 함수이다.


def evaluate_performance(y_true, y_pred):
    """분류 리포트를 만들어 출력합니다."""
    # y_true : 실제 정답 라벨 (0 또는 1)
    # y_pred : 모델이 예측한 라벨 (0 또는 1)
    # 두 리스트를 비교하여 성능 지표를 계산한다.

    performance = classification_report(
        y_true,
        y_pred,
        target_names=["Negative Review", "Positive Review"]
        # target_names는 각 숫자 라벨(0, 1)이 어떤 클래스를 의미하는지 이름을 붙여주는 옵션이다.
        # 0 → "Negative Review"
        # 1 → "Positive Review"
    )

    print(performance)
    # 계산된 성능표를 화면에 출력한다.

In [8]:
evaluate_performance(data["test"]["label"], y_pred)
# data["test"]["label"] :
#   Rotten Tomatoes 테스트 데이터셋에 들어 있는 실제 정답(label) 값들이다.
#   0은 부정(negative), 1은 긍정(positive)을 의미한다.

# y_pred :
#   우리가 만든 감정 분석 모델이 예측한 결과 리스트.
#   앞에서 만들어둔 y_pred에는 0(부정) 또는 1(긍정)이 순서대로 저장되어 있다.

# evaluate_performance 함수는
#   실제 정답(y_true)과 예측값(y_pred)을 비교하여
#   정확도(precision, recall, f1-score)를 출력해주는 기능을 한다.

# 즉, 이 코드는 모델 성능을 최종적으로 평가하는 단계이다.

                 precision    recall  f1-score   support

Negative Review       0.76      0.88      0.81       533
Positive Review       0.86      0.72      0.78       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



# 임베딩을 활용하여 분류 작업 수행하기

### Sentence-BERT(문장 임베딩 모델)의 역할
- 문장을 “의미 기반 벡터”로 변환해 유사한 문장끼리는 가까운 거리,
- 다른 의미의 문장끼리는 먼 거리가 되도록 매핑한다.

### 왜 SBERT를 사용할까?

- 문장 의미 이해 능력이 뛰어남

- 검색·분류·QA·챗봇 등 활용도가 매우 높음

- 기존 BERT보다 속도·성능이 균형 잡혀 있음

## 지도 학습 분류

In [9]:
from sentence_transformers import SentenceTransformer
# SentenceTransformer는 문장(텍스트)을 숫자 벡터로 변환(임베딩)해주는 라이브러리이다.
# BERT 기반으로 문장의 의미를 잘 반영한 벡터를 만들어 준다.
# 검색, 유사도 계산, 분류 모델의 입력 등에 많이 사용된다.

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
# 'all-mpnet-base-v2'는 SBERT 계열의 대표적인 문장 임베딩 모델이다.
# 문장 의미를 잘 표현해 주기 때문에 문서 검색, 감정 분석, QA 등에서 널리 사용된다.

# 텍스트를 임베딩으로 변환합니다.
train_embeddings = model.encode(list(data["train"]["text"]), show_progress_bar=True)
# data["train"]["text"] : 학습 데이터의 텍스트 목록
# encode(...) : 텍스트를 768차원의 벡터로 변환한다.
# show_progress_bar=True : 진행 상황을 표시(훈련 데이터가 많을 때 유용)

test_embeddings = model.encode(list(data["test"]["text"]), show_progress_bar=True)
# 테스트 데이터도 같은 방식으로 벡터화한다.
# 이렇게 생성된 임베딩은 머신러닝 모델(SVM, 로지스틱 회귀 등) 또는
# 벡터 유사도 기반 분류에 활용할 수 있다.

Batches:   0%|          | 0/267 [00:00<?, ?it/s]

Batches:   0%|          | 0/34 [00:00<?, ?it/s]

In [11]:
train_embeddings.shape
# train_embeddings는 학습 데이터(train 텍스트)를 임베딩한 결과이다.
# shape는 이 임베딩 배열의 "크기(행, 열)"를 보여준다.

# 예를 들어 (8530, 768)이 출력되었다면:
#  - 8530 : 학습 데이터 문장 개수
#  - 768  : 각 문장을 나타내는 임베딩 벡터의 차원(길이)
#
# 즉, 학습 데이터의 모든 문장이
# 768차원짜리 숫자 벡터로 변환되었다는 의미이다.

(8530, 768)

왜 로지스틱 회귀를 사용할까?

- 간단하고 빠르며, 임베딩 기반 분류에서 성능이 잘 나오는 편

- 모델 구조가 단순해서 이해하기 좋음

- SBERT 같은 고성능 임베딩을 사용하면 이 단순 모델로도 높은 정확도 달성 가능

학습되는 내용은 무엇인가?

- 768차원 벡터를 입력받아 0(부정) 혹은 1(긍정) 중 하나로 분류하는 규칙을 학습하는 것

- 경계(boundary)를 찾아 각 문장을 어떤 그룹으로 나눌지 결정

In [12]:
from sklearn.linear_model import LogisticRegression
# LogisticRegression은 머신러닝에서 가장 기본적인 분류 모델 중 하나이다.
# 입력된 벡터(임베딩)를 보고 0 또는 1 같은 이진 라벨을 예측하는 데 자주 사용된다.

# 훈련 세트의 임베딩으로 로지스틱 회귀 모델을 훈련합니다.
clf = LogisticRegression(random_state=42)
# clf는 분류 모델(classifier)을 의미한다.
# random_state=42는 실험을 반복해도 같은 결과가 나오도록 설정한 값이다.

clf.fit(train_embeddings, data["train"]["label"])
# fit(입력값, 정답값) : 모델을 학습시키는 과정
# train_embeddings       → SBERT가 만든 문장 임베딩(각 문장은 768차원 벡터)
# data["train"]["label"] → 해당 문장이 긍정(1)인지 부정(0)인지 라벨 정보

# 즉, 이 코드는 "문장 벡터 → 감정(긍정/부정)"을 예측할 수 있는 분류 모델을 만든 것이다.

LogisticRegression(random_state=42)

이 단계가 하는 일은?
- “학습한 모델이 새 문장을 얼마나 정확하게 분류하는지 확인하는 과정”

왜 임베딩을 입력으로 쓰는가?
- SBERT가 문장을 의미 기반 벡터로 변환했기 때문에
- 로지스틱 회귀처럼 단순한 모델로도 높은 성능을 낼 수 있다.

In [13]:
# 테스트 세트 임베딩에 대해 예측을 수행합니다.
y_pred = clf.predict(test_embeddings)
# clf.predict(...) :
#   학습된 로지스틱 회귀 모델(clf)이 테스트 데이터의 임베딩을 보고
#   각 문장이 긍정(1)인지 부정(0)인지 예측하는 단계이다.
#
# test_embeddings : SBERT로 만든 테스트 데이터 문장 임베딩(768차원 벡터)
# y_pred          : 예측 결과가 0 또는 1의 리스트 형태로 저장된다.


# 모델 성능을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)
# data["test"]["label"]은 테스트 데이터의 실제 정답(0 또는 1)
# y_pred는 모델이 예측한 값
#
# evaluate_performance 함수는 두 값을 비교해서
# Precision, Recall, F1-score 등을 출력해준다.
# 즉, 모델이 얼마나 잘 맞췄는지 확인하는 단계이다.

                 precision    recall  f1-score   support

Negative Review       0.85      0.86      0.85       533
Positive Review       0.86      0.85      0.85       533

       accuracy                           0.85      1066
      macro avg       0.85      0.85      0.85      1066
   weighted avg       0.85      0.85      0.85      1066



**팁!**  

분류기를 사용하지 않는다면 어떻게 할 수 있을까요? 분류기 대신 클래스별 임베딩을 평균하고 코사인 유사도를 적용하여 문서와 가장 잘 맞는 클래스를 예측할 수 있습니다.

### 아래 방식은 “평균 임베딩 기반 분류(Nearest Centroid Classifier)”라고 한다.

- 각 감정(부정/긍정)에 대한 대표 벡터(centroid)를 구하고
- 테스트 문장이 어떤 대표 벡터와 더 가까운지 비교하여 분류하는 방식.

장점

- 모델 학습이 필요 없음

- 빠르고 단순

단점

- 데이터 분포가 복잡하면 정확도 낮아짐

- 한 점(centroid)으로 전체 문장을 대표해야 하는 한계

In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity : 두 벡터가 얼마나 비슷한지(각도가 얼마나 가까운지) 계산하는 함수이다.
# 값이 1에 가까울수록 매우 비슷한 문장이라는 뜻이다.


# ---------------------------------------------------------
# 1) 레이블별(부정/긍정) 평균 임베딩 만들기
# ---------------------------------------------------------

# train_embeddings : 학습 데이터 문장들의 임베딩(예: (8529, 768) 크기)
# data["train"]["label"] : 각 문장이 긍정(1)인지 부정(0)인지 정답 라벨

# np.hstack([...]) :
#   임베딩 벡터 뒤에 라벨(0 또는 1)을 하나 붙여서 하나의 큰 배열로 만든다.
# reshape(-1, 1) : 라벨을 열(컬럼) 형태로 만들어 붙이기 위함.
df = pd.DataFrame(
    np.hstack([
        train_embeddings,
        np.array(data["train"]["label"]).reshape(-1, 1)
    ])
)

# 이제 df는 769개의 컬럼을 가진다.
# 0~767번 컬럼 → 문장 임베딩
# 768번 컬럼    → 레이블(0=부정, 1=긍정)

# df.groupby(768).mean() :
#   레이블(0 또는 1)을 기준으로 그룹을 나누고
#   각 그룹의 임베딩을 평균낸다.
#   즉, “부정 문장들의 평균 벡터”, “긍정 문장들의 평균 벡터”를 만든다.
averaged_target_embeddings = df.groupby(768).mean().values
# 결과 shape : (2, 768)
# → 0번 행 = 부정 문장 평균 벡터
# → 1번 행 = 긍정 문장 평균 벡터


# ---------------------------------------------------------
# 2) 테스트 임베딩과 평균 임베딩의 유사도 비교
# ---------------------------------------------------------

# cosine_similarity(test, target)
#   test 문장 임베딩 각각을 "부정 평균 벡터", "긍정 평균 벡터"와 비교해서
#   어느 쪽과 더 비슷한지를 계산한다.
sim_matrix = cosine_similarity(test_embeddings, averaged_target_embeddings)
# sim_matrix의 크기 : (테스트 문장 수, 2)
# 예: sim_matrix[i] = [0번 클래스(부정)와의 유사도, 1번 클래스(긍정)와의 유사도]

# 각 테스트 문장마다 더 유사한 클래스를 선택한다.
y_pred = np.argmax(sim_matrix, axis=1)
# np.argmax : 가장 큰 값을 가진 인덱스(0 또는 1)를 선택한다.
# → 예측 결과(0=부정, 1=긍정)


# ---------------------------------------------------------
# 3) 모델 성능 평가
# ---------------------------------------------------------

# 실제 라벨과(y_true) 예측 라벨(y_pred)을 비교하여 성능을 출력한다.
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.85      0.84      0.84       533
Positive Review       0.84      0.85      0.84       533

       accuracy                           0.84      1066
      macro avg       0.84      0.84      0.84      1066
   weighted avg       0.84      0.84      0.84      1066



## 데이터에 레이블이 없는 경우

In [15]:
# 레이블의 임베딩을 만듭니다.
label_embeddings = model.encode(["A negative review", "A positive review"])
# 여기서는 각 레이블(부정, 긍정)을 짧은 문장으로 표현하고
# 그 문장을 SBERT 모델을 이용해 임베딩(숫자 벡터)으로 변환한다.
#
# "A negative review"  → 부정 리뷰를 나타내는 문장
# "A positive review"  → 긍정 리뷰를 나타내는 문장
#
# model.encode(...) :
#   문장을 의미 기반 768차원 벡터로 바꿔주는 함수이다.
#
# label_embeddings는 레이블 2개를 각각 벡터로 만든 결과이며
# shape는 (2, 768)이 된다.
#
# 이후에는 테스트 문장이
# 이 두 벡터 중 어느 것과 더 가까운지(코사인 유사도) 비교해서
# 부정/긍정을 예측하는 데 사용한다.

SBERT가 문장 의미를 벡터로 표현한다.

레이블 자체를 문장으로 표현해 임베딩한다.

테스트 문장이 어떤 레이블 벡터와 더 가까운지를 비교하여 분류한다.

이는 코사인 유사도 기반의 매우 직관적이고 간단한 “문장 의미 매칭 방식”이다.

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity :
#   두 벡터가 얼마나 비슷한지(각도가 얼마나 가까운지) 계산하는 함수이다.
#   값이 1에 가까울수록 두 문장의 의미가 비슷하다는 뜻이다.

# 각 문서(테스트 문장)와 가장 잘 맞는 레이블을 찾습니다.
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
# test_embeddings : 테스트 데이터 문장들을 768차원 벡터로 변환한 것
# label_embeddings :
#   "A negative review", "A positive review" 문장을 SBERT로 임베딩한 벡터
#
# cosine_similarity(test_embeddings, label_embeddings)은
#   각 테스트 문장이 두 레이블 벡터(부정, 긍정)와 각각 어느 정도 비슷한지 계산한다.
#
# 결과 sim_matrix의 크기 → (테스트 문장 수, 2)
# 예: sim_matrix[i] = [부정 레이블과의 유사도, 긍정 레이블과의 유사도]


y_pred = np.argmax(sim_matrix, axis=1)
# np.argmax : 가장 큰 값을 가진 인덱스를 선택하는 함수
# 즉, 부정/긍정 중 어떤 레이블과 더 비슷한지를 선택한다.
#
# sim_matrix[i]가 [0.32, 0.78]이라면 → 긍정(1)에 더 가깝기 때문에 1로 예측
# y_pred : 모든 테스트 문장에 대해 0(부정), 1(긍정)으로 이루어진 예측 결과 리스트

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.78      0.77      0.78       533
Positive Review       0.77      0.79      0.78       533

       accuracy                           0.78      1066
      macro avg       0.78      0.78      0.78      1066
   weighted avg       0.78      0.78      0.78      1066



**팁!**  

설명을 다르게 하면 어떨까요? **"A very negative movie review"**와 **"A very positive movie review"**를 사용하면 어떻게 되는지 확인해 보세요!

레이블을 간단한 단어 대신 문장 형태로 만들면
SBERT가 의미를 더 잘 파악해 분류 성능이 오르는 경우가 있다.

이 방법은 “문장 의미 기반 분류”로,
테스트 문장이 두 레이블 문장 중 어떤 것과 더 의미적으로 가까운지를 비교해 분류한다.

모델 학습이 필요하지 않은 매우 직관적이고 빠른 방식이다.

In [17]:
# 레이블(부정/긍정)을 더 분명하게 표현한 문장을 임베딩합니다.
label_embeddings = model.encode([
    "A very negative movie review",   # 매우 부정적인 영화 리뷰
    "A very positive movie review"    # 매우 긍정적인 영화 리뷰
])
# 문장을 SBERT 모델에 넣어 임베딩(768차원 벡터)으로 변환합니다.
# 레이블을 더 구체적이고 긴 문장으로 만들면
# 문장 의미가 더 분명해져 유사도 계산에서 도움이 될 때가 있다.


# 테스트 문장 임베딩과 레이블 임베딩의 유사도를 계산합니다.
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
# test_embeddings : 테스트 데이터 문장들의 임베딩
# label_embeddings : "아주 부정적/아주 긍정적" 문장을 임베딩한 두 벡터
#
# sim_matrix[i] = [부정 레이블과의 유사도, 긍정 레이블과의 유사도]


# 테스트 문장이 어떤 레이블과 더 비슷한지 예측합니다.
y_pred = np.argmax(sim_matrix, axis=1)
# np.argmax는 가장 높은 유사도를 가진 인덱스를 선택한다.
# 0 → 부정, 1 → 긍정


# 모델 성능을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)
# 실제 정답(label)과 예측 결과(y_pred)를 비교하여
# precision, recall, f1-score 등을 출력한다.

                 precision    recall  f1-score   support

Negative Review       0.86      0.73      0.79       533
Positive Review       0.76      0.88      0.82       533

       accuracy                           0.80      1066
      macro avg       0.81      0.80      0.80      1066
   weighted avg       0.81      0.80      0.80      1066



# 생성 모델로 텍스트 분류하기

## T5 모델 사용하기

FLAN-T5
- 구글이 공개한 고성능 “지시어 기반(Instruct)” 언어 모델
- 명령형(prompt) 입력에 매우 민감하고, 작은 모델도 성능이 우수하다.

text2text-generation 파이프라인
- 어떤 입력이 들어와도 “새로운 텍스트 출력”으로 해결하는 범용 모델 방식

In [18]:
# 모델을 로드합니다.
pipe = pipeline(
    "text2text-generation",   # FLAN-T5는 모든 작업을 "텍스트 → 텍스트" 형태로 처리하는 모델이다.
                              # 즉, 요약, 번역, 질의응답 등 거의 모든 작업을
                              # 입력 문장을 보고 새로운 문장으로 출력한다.

    model="google/flan-t5-small",  # 사용할 사전학습 모델 이름.
                                   # FLAN-T5-small은 가벼운 크기의 T5 모델로,
                                   # 여러 NLP 작업을 명령어(prompt) 기반으로 수행할 수 있다.

    device="cuda:0"                # GPU 0번을 사용하여 더 빠르게 추론한다.
)
# pipeline을 사용하면 모델과 토크나이저 설정을 직접 하지 않아도 되고,
# pipe("문장 입력") 형태로 바로 텍스트 생성 작업을 처리할 수 있다.

Device set to use cuda:0


T5·FLAN-T5 모델은 “문장 그대로 넣으면 감정 분석을 하지 않는다.”
- 모델이 어떤 작업을 해야 하는지 명시적으로 지시(prompt) 해야 한다.

프롬프트 예시

- Is the following sentence positive or negative? <리뷰 문장>


이렇게 하면 모델이
- “이 문장의 감정이 긍정인지 부정인지 판단하라”
- 라는 의도를 정확히 이해하고 답을 생성한다.

In [19]:
# 프롬프트를 추가합니다.
prompt = "Is the following sentence positive or negative? "
# FLAN-T5는 "지시어(prompt)"를 통해 어떤 작업을 수행할지 알려주어야 한다.
# 여기서는 감정 분석(Sentiment Classification)을 하기 위해
# 문장 앞에 질문 형태의 프롬프트를 붙여주는 방식이다.
# 예:
#   "Is the following sentence positive or negative? This movie was great!"


# data.map(...) :
#   데이터셋의 각 항목(example)에 대해 새로운 필드를 추가해주는 함수이다.
#   여기서는 각 문장(example['text']) 앞에 prompt 문장을 붙여서
#   't5'라는 새로운 컬럼을 만드는 작업을 한다.
data = data.map(lambda example: {"t5": prompt + example['text']})
# 예를 들어, 원래 example['text']가
#   "The movie was amazing!"
# 이라면,
#   example['t5'] = "Is the following sentence positive or negative? The movie was amazing!"


data
# 프롬프트가 잘 추가된 새로운 데이터셋 구조를 출력해 확인한다.

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

FLAN-T5는 문장을 분류하는 모델이 아니라 “문장 생성 모델”이다.
- 감정 분석도 “답을 텍스트로 생성”하는 방식으로 수행한다.

그래서 실제 정답은 숫자가 아니라

- "positive", "negative"
- 같은 텍스트 형태로 나온다.

우리는 이를 0 = 부정, 1 = 긍정 으로 변환해 평가에 적합한 형태로 만든다.

In [20]:
# 추론을 실행합니다.
y_pred = []

# pipe(KeyDataset(...)) :
#   테스트 데이터에서 "t5" 컬럼만 꺼내어 FLAN-T5 모델에 순서대로 넣는다.
# tqdm(...) :
#   처리 진행 상황을 progress bar로 보여주는 기능.

for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
    # output은 FLAN-T5의 생성 결과이다.
    # 예: [{'generated_text': 'positive'}] 또는 [{'generated_text': 'negative'}]

    text = output[0]["generated_text"]
    # 모델이 출력한 텍스트(positive 또는 negative)를 가져온다.

    # 모델이 "negative"를 출력하면 0, 그 외에는 1로 분류한다.
    # ex) text == "negative" → 0
    #     text == "positive" → 1
    y_pred.append(0 if text == "negative" else 1)
    # 이렇게 해서 감정 예측 결과를 리스트에 차례대로 저장한다.

100%|██████████| 1066/1066 [01:16<00:00, 14.02it/s]


data["test"]["label"] = 실제 감정(0/1)

y_pred = 모델이 예측한 감정(0/1)

두 개를 비교해서 모델이 얼마나 정확했는지 알려준다.

In [21]:
evaluate_performance(data["test"]["label"], y_pred)
# data["test"]["label"] :
#   테스트 데이터의 실제 정답(부정 = 0, 긍정 = 1)

# y_pred :
#   우리가 만든 FLAN-T5 모델이 예측한 결과(0 또는 1)

# evaluate_performance(정답, 예측값) :
#   두 값을 비교하여 모델의 성능을 출력하는 함수이다.
#
# Precision(정밀도), Recall(재현율), F1-score 같은
# 분류 모델의 핵심 지표들을 표로 보여준다.
#
# 즉, "FLAN-T5 감정 분석 모델이 얼마나 잘 맞췄는지"
# 마지막으로 평가하는 단계이다.

                 precision    recall  f1-score   support

Negative Review       0.83      0.85      0.84       533
Positive Review       0.85      0.83      0.84       533

       accuracy                           0.84      1066
      macro avg       0.84      0.84      0.84      1066
   weighted avg       0.84      0.84      0.84      1066



## ChatGPT로 분류하기

openai 최신 버전과 httpx 최신 버전은 가끔 호환성 문제가 생긴다.

특히 OpenAI API 호출 시 proxies= 옵션 관련 오류가 발생하기도 한다.

그래서 정상 작동이 확인된 버전 조합으로 재설치하는 것이 가장 확실한 해결책이다.

--force-reinstall은 이미 설치되어 있어도 다시 설치하게 하는 옵션이다.

In [22]:
# httpx 패키지의 proxies 매개변수 오류를 피하기 위해 아래와 같이 버전을 고정해 설치합니다.
# 참고: openai(1.56.0 이상) + httpx 최신 버전 조합에서
#       "unexpected keyword argument 'proxies'" 오류가 발생하는 문제가 있다.
# 해결 방법: openai와 httpx를 서로 호환되는 버전으로 재설치한다.

# --force-reinstall : 이미 설치된 버전이 있어도 강제로 다시 설치한다.
# --quiet            : 설치 메시지를 화면에 많이 출력하지 않도록 한다.

!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4

OpenAI API 사용 = “OpenAI 서버에 요청을 보내고, 모델 답변을 받는 것”

이를 위해 client 객체가 필요함

API 키는 사용자를 인증하는 비밀번호 같은 역할

키가 없다면 모델을 사용할 수 없음

In [ ]:
import openai
# openai 패키지는 OpenAI API(GPT 모델 등)를 파이썬에서 사용할 수 있게 해주는 라이브러리이다.


# 클라이언트를 만듭니다.
client = openai.OpenAI(api_key="YOUR_KEY_HERE")
# OpenAI API를 사용하려면 "클라이언트(client)" 객체를 먼저 만들어야 한다.
# 이 클라이언트를 통해 서버에 요청을 보내고, 모델의 응답을 받아올 수 있다.

# api_key="YOUR_KEY_HERE" :
#   OpenAI 계정에서 발급받은 개인 API 키를 입력해야 한다.
#   이 키는 본인만 사용해야 하므로 절대 노출되면 안 된다.
#
# 이후부터는 아래처럼 사용할 수 있다.
#   client.responses.create(...)
#   client.chat.completions.create(...)
#
# 즉, client는 OpenAI 모델들과 대화하는 '연결 통로' 역할을 한다.

ChatGPT API는 “대화를 주고받는 구조”로 되어 있다.

messages=[...] 안에 system/user 역할을 담아 모델에게 전달한다.

모델의 출력은
chat_completion.choices[0].message.content
에 저장된다.

temperature=0은 “정확히 하나의 정답을 원할 때” 주로 설정한다.

In [ ]:
def chatgpt_generation(prompt, document, model="gpt-4o-mini"):
    """프롬프트와 문서를 입력받아 ChatGPT로부터 출력을 생성하는 함수입니다."""

    # messages 리스트는 ChatGPT에게 전달할 대화 형식의 입력이다.
    # system : 모델의 역할 또는 말투 등을 설정
    # user   : 실제 사용자가 입력한 메시지
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            # 모델에게 “도움이 되는 조수” 역할을 부여한다.
        },
        {
            "role": "user",
            "content": prompt.replace("[DOCUMENT]", document)
            # prompt 안의 [DOCUMENT] 부분을 실제 문서 내용으로 교체하여 전달한다.
            # 예: prompt = "Summarize this: [DOCUMENT]"
            #     document = "This movie was great."
            #     → "Summarize this: This movie was great."
        }
    ]

    # 모델에게 메시지를 보내고 응답을 받는 부분
    chat_completion = client.chat.completions.create(
        messages=messages,   # 위에서 만든 대화 입력
        model=model,         # 사용할 LLM (기본값: gpt-4o-mini)
        temperature=0        # 출력의 랜덤성을 낮춰 항상 같은 결과가 나오도록 한다
    )

    # 모델이 생성한 답변 텍스트만 꺼내서 반환한다.
    # choices[0] : 여러 답변 중 첫 번째 선택지
    # message.content : 실제 생성된 텍스트
    return chat_completion.choices[0].message.content

In [ ]:
# 프롬프트 템플릿을 정의합니다.
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""
# 위 프롬프트는 GPT에게 “문서가 긍정인지 부정인지 예측하라”고 지시하는 문장이다.
# [DOCUMENT]라는 부분을 나중에 실제 리뷰 문장으로 교체하여 사용한다.
# 출력은 반드시 1(긍정) 또는 0(부정)만 허용하도록 명확히 지시하고 있다.


# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
# 감정 분석할 실제 영화 리뷰 문장이다.
# 예: "꾸밈없고, 매력적이며, 유쾌하고, 독창적이다." → 매우 긍정적인 표현

chatgpt_generation(prompt, document)
# chatgpt_generation 함수는
#   1) prompt 안에 [DOCUMENT]를 실제 문서로 바꾸고
#   2) GPT 모델(gpt-4o-mini)에 전달한 뒤
#   3) 생성된 답변(0 또는 1)을 반환한다.

# 이 문장의 경우 긍정적인 표현이므로 GPT는 보통 "1"을 반환한다.

'1'

다음 단계는 전체 테스트 데이터로 오픈AI 모델 중 하나를 실행하는 것입니다. 하지만 전체 테스트 데이터는 1,066개이므로 충분한 크레딧이 있을 때만 실행하세요.

In [ ]:
# 무료 크레딧을 아끼고 싶다면 이 코드를 건너 뛰세요.
# 이유: 테스트 데이터 전체에 대해 GPT를 한 번씩 호출하면
#       호출 횟수가 매우 많아져 비용이 커질 수 있다.

# predictions 리스트에 GPT 예측 결과를 모두 저장합니다.
predictions = [
    chatgpt_generation(prompt, doc)      # 한 문서를 넣어 GPT에게 긍/부정 예측 요청
    for doc in tqdm(data["test"]["text"]) # tqdm : 진행 상황(progress bar) 표시
]
# data["test"]["text"] : 테스트용 리뷰 문장들의 리스트
# chatgpt_generation() : GPT에 프롬프트 + 문서를 전달하여 0 또는 1을 받아오는 함수

# 결국 predictions에는
#   ["1", "0", "1", "1", ...] 와 같은 예측 결과가 순서대로 저장된다.

100%|██████████| 1066/1066 [08:54<00:00,  2.00it/s]


GPT는 "0" 또는 "1"을 문자열(str) 로 반환한다.

평가하려면 반드시 정수(int)로 바꿔야 한다.

이후 실제 정답과 비교해 성능을 계산한다.

In [ ]:
# 예측을 저장합니다.
y_pred = [int(pred) for pred in predictions]
# ChatGPT의 출력(pred)은 문자열 형태("0" 또는 "1")로 반환된다.
# 머신러닝 평가 함수는 정수형 라벨(0, 1)을 요구하므로
# int(pred)를 사용해 문자열 → 정수로 변환한다.
# 결과적으로 y_pred는 [0, 1, 1, 0, ...] 형태의 리스트가 된다.


# 성능을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)
# data["test"]["label"] : 테스트 데이터의 실제 정답
# y_pred                : GPT가 예측한 정답

# evaluate_performance 함수는
#   precision, recall, f1-score 등을 출력하여
#   모델 성능을 종합적으로 평가한다.

                 precision    recall  f1-score   support

Negative Review       0.88      0.96      0.92       533
Positive Review       0.96      0.87      0.91       533

       accuracy                           0.92      1066
      macro avg       0.92      0.92      0.92      1066
   weighted avg       0.92      0.92      0.92      1066

